In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy.util as util
import pandas as pd
import numpy as np

In [3]:
ID = "3dce0b0ef6a442c9878c48e689d70df7"
SECRET = "9a97c11430d04273ad190cab400479eb"

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=ID, client_secret=SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Links to Playlist


In [6]:
links = {3 : "37i9dQZF1DXdPec7aLTmlC", 4: "37i9dQZF1DX7qK8ma5wgG1", 0 : "4pmAr4R0W5b23fKjUfnwqU", 6 : "6f9nZmbYGY8jnoR5QTrSz4", 2 : "5AM4lgcUAw5sokybXj3ny7"}

user_ids = {3 : "spotify", 4 : "spotify", 0 : "aravind", 6 : "yogi", 2 : "glitterleopard"}

In [7]:
playlist_link = "https://open.spotify.com/playlist/4pmAr4R0W5b23fKjUfnwqU?si=ceac770447ad4c37"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [8]:
sp.user_playlist_tracks(user_ids[0], links[0])

{'href': 'https://api.spotify.com/v1/playlists/4pmAr4R0W5b23fKjUfnwqU/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2023-03-03T00:36:56Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/f9n8d6eqvtou7bj3lo6kt29qj'},
    'href': 'https://api.spotify.com/v1/users/f9n8d6eqvtou7bj3lo6kt29qj',
    'id': 'f9n8d6eqvtou7bj3lo6kt29qj',
    'type': 'user',
    'uri': 'spotify:user:f9n8d6eqvtou7bj3lo6kt29qj'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5ja9cfKWW8Zj9aloyK8WuY'},
       'href': 'https://api.spotify.com/v1/artists/5ja9cfKWW8Zj9aloyK8WuY',
       'id': '5ja9cfKWW8Zj9aloyK8WuY',
       'name': 'MoonDeity',
       'type': 'artist',
       'uri': 'spotify:artist:5ja9cfKWW8Zj9aloyK8WuY'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'A

In [9]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist", "album", "track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [10]:
df = pd.DataFrame()
for key in links:
    fd = analyze_playlist(user_ids[key], links[key])
    df = df.append(fd)

C:\Users\arvin\AppData\Local\Temp\ipykernel_13128\3329711457.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(fd)
C:\Users\arvin\AppData\Local\Temp\ipykernel_13128\3329711457.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(fd)
C:\Users\arvin\AppData\Local\Temp\ipykernel_13128\3329711457.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(fd)
C:\Users\arvin\AppData\Local\Temp\ipykernel_13128\3329711457.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(fd)
C:\Users\arvin\AppData\Local\Temp\ipykernel_13128\3329711457.py:4: FutureWarning: The frame.append method is depreca

In [11]:
df

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,David Guetta,I'm Good (Blue),I'm Good (Blue),4uUG5RXrOk84mYEfFvj3cK,0.561,0.965,7,-3.673,0,0.0343,0.000007,0.3710,0.304,128.040,175238,4
1,OneRepublic,I Ain’t Worried (Music From The Motion Picture...,I Ain't Worried,4h9wh7iOZ0GGn8QVp4RAOB,0.704,0.797,0,-5.927,1,0.0475,0.000745,0.0546,0.825,139.994,148486,4
2,Meghan Trainor,Takin' It Back,Made You Look,0QHEIqNKsMoOY5urbzN48u,0.838,0.525,10,-3.562,1,0.0665,0.000002,0.0771,0.884,144.981,134256,4
3,Nicky Youre,Shut Me Up,Shut Me Up,1XuushYZmiAuKpPYTj7p3v,0.623,0.641,10,-3.987,1,0.0462,0.000001,0.0600,0.735,185.888,140710,4
4,Coi Leray,Players,Players,6UN73IYd0hZxLi8wFPMQij,0.954,0.516,6,-5.817,1,0.1600,0.000008,0.0504,0.624,105.001,139560,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Set It Off,Cinematics (Expanded Edition),Partners in Crime (feat. Ash Costello),4oaU0fMSg3n9kqOwmLPVhH,0.376,0.939,1,-3.437,1,0.0572,0.0,0.1160,0.815,190.109,211027,4
75,Cage The Elephant,Cage The Elephant (Expanded Edition),Ain't No Rest for the Wicked,3Pzh926pXggbMe2ZpXyMV7,0.636,0.849,0,-7.075,1,0.1060,0.0,0.3720,0.917,156.036,175493,4
76,My Chemical Romance,The Black Parade,Teenagers,7j31rVgGX9Q2blT92VBEA0,0.463,0.857,4,-3.063,1,0.0632,0.0,0.1840,0.856,111.647,161920,4
77,Various Artists,Spider-Man: Into the Spider-Verse (Soundtrack ...,What's Up Danger (with Black Caviar),5zsHmE2gO3RefVsPyw2e3T,0.701,0.755,2,-5.820,1,0.0417,0.000795,0.1210,0.116,95.036,222307,4
